In [ ]:
import requests
from bs4 import BeautifulSoup as bs4
import re
import pickle

# Chrome 60 user agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36'}

data = {}
data['years'] = {}
data['errors'] = []

def displayError(errorString):
    print(errorString)
    data['errors'].append(errorString)

for sch in range(115,420): 
    sched = str(sch)
    url = 'https://www.utahutes.com/schedule.aspx?schedule='+sched
    page=requests.get(url, headers=header)
    if page.status_code == 200:
        parsed=bs4(page.text, 'html.parser')
        scheduleTable = parsed.find('article', {'class', 'sidearm-schedule'})
        scheduleType = scheduleTable.find('h2')
        if 'Football' in scheduleType.text:
            year = re.search('([0-9]+).*',scheduleType.text).group(1)
            data['years'][year] = {}
            data['years'][year]['page'] = sched
            data['years'][year]['games'] = []
            print(year)
            for game in scheduleTable.find_all('li', {'class': 'sidearm-schedule-game'}):
                gameRow = game.find('div', {'class': 'sidearm-schedule-game-row'})
                opponent = gameRow.find('div', {'class': 'sidearm-schedule-game-opponent'})
                details = opponent.find('div', {'class': 'sidearm-schedule-game-opponent-details'})
                gameDateHtml = details.find('div', {'class': 'sidearm-schedule-game-opponent-date'}).find('span')
                gameDate = re.search('([a-zA-Z]+ [0-9]+) .*', gameDateHtml.text.strip()).group(1)
                oppHtml = details.find('div', {'class': 'sidearm-schedule-game-opponent-text'})
                oppName = oppHtml.find('span', {'class': 'sidearm-schedule-game-opponent-name'})
                detailsHtml = gameRow.find('div',{'class': 'sidearm-schedule-game-details'})
                resultHtml = detailsHtml.find('div', {'class': 'sidearm-schedule-game-result'})
                if resultHtml:
                    if len(resultHtml.contents) == 9:
                        result = re.search('(.*),',resultHtml.contents[3].text).group(1)
                        capture = re.search('(.*)\-(.*)',resultHtml.contents[5].text)
                        utahScore = capture.group(1)
                        oppScore = capture.group(2)
                    else:
                        displayError(year+': Missing result for '+oppName.text.strip())
                        result = 'W'
                        utahScore = 'N/A'
                        oppScore = 'N/A'
                    game = {}
                    game['year'] = year
                    game['date'] = gameDate.zfill(2)
                    game['opponent'] = oppName.text.strip()
                    game['result'] = result
                    game['for'] = utahScore
                    game['against'] = oppScore
                    data['years'][year]['games'].append(game)
                else:
                    displayError(year+': Missing result HTML for '+oppName.text.strip())

pickle.dump(data,open('utah_all.p', 'wb'))
print(data['years']['2004'])
print(data['errors'])


In [ ]:
import sys
import pickle

alldata = pickle.load(open('utah_all.p', 'rb'))
print(alldata['years']['2004'])


In [ ]:
import csv
import pickle
from datetime import datetime


alldata = pickle.load(open('utah_all.p', 'rb'))
with open('utah_all.csv', 'wb') as csvfile:
    fieldnames = ['YEAR','DATE', 'OPPONENT', 'RESULT', 'FOR', 'AGAINST']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    writer.writeheader()
    for y,gameInfo in alldata['years'].items():
        games = gameInfo['games']
        for game in games:
            pdate = datetime.strptime(game['date'], '%b %d')
            finalDate = pdate.strftime('%m/%d/')+game['year']
            writer.writerow({
                'YEAR': game['year'], 
                'DATE': finalDate, 
                'OPPONENT': game['opponent'], 
                'RESULT': game['result'], 
                'FOR': game['for'],
                'AGAINST': game['against']
            })

In [1]:
import pickle

allgames = pickle.load(open('utah_all.p', 'rb'))

if not 'opponents' in allgames:
    allgames['opponents'] = {}
    
for year,yearInfo in allgames['years'].items():
    for game in yearInfo['games']:
        opponent = game['opponent']
        
        if not opponent in allgames['opponents']:
            print('adding '+opponent)
            allgames['opponents'][opponent] = {}
            allgames['opponents'][opponent]['games'] = []
            
        allgames['opponents'][opponent]['games'].append(game)
    print(year)

# print(allgames['opponents']['Brigham Young']['games'])


adding USC
adding Idaho
adding Arizona
adding BYU
adding Denver
adding Wyoming
adding Colorado
adding Colorado State
adding Oregon State
adding Utah State
1948
adding Washington
adding Pacific
1949
adding Santa Clara
1942
adding Fort Warren
adding Nevada
adding Tulsa
adding Colorado College
adding Saint Mary's
1943
1940
1941
adding New Mexico
adding San Francisco
adding Hawaii All-Stars
adding Hawai'i
1946
1947
adding Idaho State
1944
adding Oklahoma State
1945
adding Utah Spring Football
adding Michigan
adding Fresno State
adding Oregon
adding California
adding Arizona State
adding UCLA
2015
adding Washington State
adding Stanford
adding Red/White Spring Game
2014
adding Southern Utah
adding Brigham Young
adding San Jose State
adding Indiana
2016
adding Montana State
adding Pittsburgh
adding Georgia Tech
2011
adding UNLV
adding Iowa State
adding Air Force
adding TCU
adding Notre Dame
adding San Diego State
adding Boise State (MAACO Bowl)
2010
adding Weber State
adding Oregon State (Ho